# Гарри Поттер

### Импортирую нужные библиотеки и загружаю датасет.

In [108]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split

In [4]:
heroes = pandas.read_csv('hp-with-text.csv', sep='\t', names=["name", "house", "text"], header=1)

In [6]:
heroes.head()

,name,house,text
0,Harry Potter,Gryffindor,"Harry was an extremely brave, loyal, and selfl..."
1,Dolores Umbridge,Slytherin,Dolores Umbridge was nothing short of a sociop...
2,Horace Slughorn,Slytherin,Horace Slughorn was described as having a bumb...
3,Albus Dumbledore,Gryffindor,Considered to be the most powerful wizard of h...
4,Severus Snape,Slytherin,"At times, Snape could appear cold, cynical, ma..."


Проверяю, насколько выборка сбалансирована.

In [10]:
print(heroes.groupby('house').describe()['name'])

house             
Gryffindor  count                       42
            unique                      42
            top            James Potter II
            freq                         1
Hufflepuff  count                       11
            unique                      11
            top              Pomona Sprout
            freq                         1
Ravenclaw   count                       15
            unique                      15
            top           Helena Ravenclaw
            freq                         1
Slytherin   count                       26
            unique                      26
            top       Avery (Marauder-era)
            freq                         1
Name: name, dtype: object


Датасет насбалансирован: между Хафлпафом и Гриффиндором разница почти в 4 раза. При этом данных не так много, чтобы выкидывать какую-то их часть. С другой стороны, превосходство в 4 раза не такое большое, чтобы всё испортить. 

Если присваивать всем людям дом Гриффиндор, мы будем правы только в 42/94 = 44.7 процентах случаев. Возьмём это  за бейзлайн.

Чтобы справиться с этой проблемой, попробую использовать оверсэмплинг.

### Разделяю выборку на train и test

In [77]:
x_train, x_test, y_train, y_test = train_test_split(heroes['text'],
                                                    heroes['house'], 
                                                    test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(75,) (19,) (75,) (19,)


In [87]:
# строю мешок слов, отрезаю стоп-слова, 
bow = CountVectorizer(stop_words='english')
bowed_messages = bow.fit_transform(x_train, y_train)

Посмотрим на словарь (не знаю зачем)

In [127]:
len(bow.vocabulary_)

453

In [89]:
# oversampling
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
x_resampled, y_resampled = ros.fit_sample(bowed_messages.toarray(), y_train)

In [131]:
y_resampled

array(['Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Gryffindor', 'Gryffindor', 'Gryffindor',
       'Gryffindor', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw',
       'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Ravenclaw', 'Sly

In [126]:
x_resampled.shape

(132, 453)

In [91]:
# теперь обучаем модель. пускай это будет Наивный Байес.

naive_model = MultinomialNB()
naive_model.fit(bowed_messages, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Посмотрим, что получилось.

In [92]:
bowed_test = bow.transform(x_test)
predicted = naive_model.predict(bowed_test)

In [93]:
print(classification_report(y_test, predicted))

             precision    recall  f1-score   support

 Gryffindor       0.47      0.78      0.58         9
 Hufflepuff       0.00      0.00      0.00         3
  Ravenclaw       0.00      0.00      0.00         1
  Slytherin       0.25      0.17      0.20         6

avg / total       0.30      0.42      0.34        19



/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [94]:
confusion_matrix(y_test, predicted)

array([[7, 0, 0, 2],
       [2, 0, 0, 1],
       [1, 0, 0, 0],
       [5, 0, 0, 1]])

Как мы видим, классификатор переобучается на самых больших классах (Гриффиноре и Слизерине), и ничего не определяет Хафлпаф и Равенкло.

In [ ]:
### Теперь попробую ограничить частотность слов.

In [95]:
# строю мешок слов, отрезаю стоп-слова, ограничиваю моксимальное кол-во вхождений
bow = CountVectorizer(stop_words='english', max_df=35, min_df=5)
bowed_messages = bow.fit_transform(x_train, y_train)

# oversampling
ros = RandomOverSampler()
x_resampled, y_resampled = ros.fit_sample(bowed_messages.toarray(), y_train)

# тренирую модель
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [96]:
# тестирую
bowed_test = bow.transform(x_test)
predicted = naive_model.predict(bowed_test)

In [97]:
print(classification_report(y_test, predicted))
confusion_matrix(y_test, predicted)

             precision    recall  f1-score   support

 Gryffindor       0.58      0.78      0.67         9
 Hufflepuff       0.67      0.67      0.67         3
  Ravenclaw       1.00      1.00      1.00         1
  Slytherin       0.67      0.33      0.44         6

avg / total       0.64      0.63      0.61        19



array([[7, 1, 0, 1],
       [1, 2, 0, 0],
       [0, 0, 1, 0],
       [4, 0, 0, 2]])

### А теперь попробуем рандомный лес

In [117]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 30)
model.fit(bowed_messages, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [118]:
# тестирую
bowed_test = bow.transform(x_test)
predicted = model.predict(bowed_test)

In [119]:
print(classification_report(y_test, predicted))
confusion_matrix(y_test, predicted)

             precision    recall  f1-score   support

 Gryffindor       0.53      0.89      0.67         9
 Hufflepuff       0.00      0.00      0.00         3
  Ravenclaw       0.00      0.00      0.00         1
  Slytherin       0.67      0.33      0.44         6

avg / total       0.46      0.53      0.46        19



/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[8, 0, 1, 0],
       [2, 0, 0, 1],
       [1, 0, 0, 0],
       [4, 0, 0, 2]])

### А теперь попробуем дерево решений

In [120]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(bowed_messages, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [123]:
# тестирую
bowed_test = bow.transform(x_test)
predicted = tree.predict(bowed_test)

In [124]:
print(classification_report(y_test, predicted))
confusion_matrix(y_test, predicted)

             precision    recall  f1-score   support

 Gryffindor       0.57      0.44      0.50         9
 Hufflepuff       0.33      0.33      0.33         3
  Ravenclaw       0.00      0.00      0.00         1
  Slytherin       0.50      0.33      0.40         6

avg / total       0.48      0.37      0.42        19



array([[4, 1, 3, 1],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 2, 2]])